In [1]:
import pandas as pd
from sklearn import preprocessing
from sklearn import metrics
from sklearn.externals import joblib
from sklearn.ensemble import RandomForestClassifier
# from gcforest.gcforest import GCForest
from GCForest import *

In [6]:
# data preprocessing
df = pd.read_csv('../data/full_size/atec_anti_fraud_train.csv',index_col = 0) # read large data
# df = df[df['label']!=-1]
# df = df.fillna(df.mode().iloc[0]) # fill nan with most common value

In [14]:
df.info(verbose=True,null_counts=True)

<class 'pandas.core.frame.DataFrame'>
Index: 994731 entries, f10eb20f31cf7063ee8bdbd1272214e4d7e0193c8dbce45ad14463036ff95d0a to 0e58a42c95d7b4d2abb495cb2c01ad90f02465488b86c5b68cafd94d65f81e1c
Data columns (total 299 columns):
label    994731 non-null int64
date     994731 non-null int64
f1       994731 non-null int64
f2       994731 non-null int64
f3       994731 non-null int64
f4       994731 non-null int64
f5       794906 non-null float64
f6       994731 non-null int64
f7       994731 non-null int64
f8       994731 non-null int64
f9       994731 non-null int64
f10      994731 non-null int64
f11      994731 non-null int64
f12      994731 non-null int64
f13      994731 non-null int64
f14      994731 non-null int64
f15      994731 non-null int64
f16      994731 non-null int64
f17      994731 non-null int64
f18      994731 non-null int64
f19      994731 non-null int64
f20      787283 non-null float64
f21      787283 non-null float64
f22      787283 non-null float64
f23      787283 non-

In [ ]:
train_num = int(0.75*df.shape[0])
train_data = df.iloc[:train_num,:] # feature without time
test_data = df.iloc[train_num:,:] # label

train_x = train_data.iloc[:,1:]
train_y = train_data.iloc[:,0]
test_x = test_data.iloc[:,1:]
test_y = test_data.iloc[:,0]

In [ ]:
# set up configuration of gcforest
def get_toy_config():
    config = {}
    ca_config = {}
    ca_config["random_state"] = 0
    ca_config["max_layers"] = 100
    ca_config["early_stopping_rounds"] = 3
    ca_config["n_classes"] = 2
    ca_config["estimators"] = []
    ca_config["estimators"].append(
            {"n_folds": 5, "type": "XGBClassifier", "n_estimators": 10, "max_depth": 5,
             "objective": "binary:logistic", "silent": False, "learning_rate": 0.1} )
    ca_config["estimators"].append({"n_folds": 5, "type": "RandomForestClassifier", "n_estimators": 10, "max_depth": None, "n_jobs": -1})
    ca_config["estimators"].append({"n_folds": 5, "type": "ExtraTreesClassifier", "n_estimators": 10, "max_depth": None, "n_jobs": -1})
    ca_config["estimators"].append({"n_folds": 5, "type": "LogisticRegression"})
    config["cascade"] = ca_config
    return config



In [8]:
# config = get_toy_config()
# gcf = GCForest(config)
# x_train_enc = gcf.fit_transform(train_x.values,train_y.values)

gcf = gcForest(shape_1X=(1,train_x.shape[1]),window=train_x.shape[1])
gcf.fit_transform(train_x.values,train_y.values)

Slicing Sequence...
Training MGS Random Forests...


e:\python352\lib\site-packages\sklearn\ensemble\forest.py:453: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
e:\python352\lib\site-packages\sklearn\ensemble\forest.py:458: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])


Adding/Training Layer, n_layer=1


ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

In [ ]:
# accuracy_test
y_test_predict = clf.predict(x_test)
print(metrics.accuracy_score(y_test,y_test_predict))

In [ ]:
# probability output for future evaluation
y_test_predict = clf.predict_proba(x_test)
result = pd.DataFrame({'score':y_test_predict[:,1],'truth':y_test})
result.to_csv('./result/wb_lr_notime.csv')

In [ ]:
# save model
model_name = 'wb_lr.pkl'
joblib.dump(clf,model_name)